In [1]:
import pandas as pd
import numpy as np

In [2]:
df1 = pd.read_csv('2024_feb_unduplicated_orders.csv')
df1['title'] = df1['title'].str.lstrip()
df1['title'] = df1['title'].str.rstrip()
df1.shape

(59185, 22)

In [3]:
df1_notnull = df1[df1['customs_description'].notnull()]
df1_null = df1[df1['customs_description'].isna()]
df1_notnull.shape, df1_null.shape, len(df1_null.isna())

((56557, 22), (2628, 22), 2628)

In [4]:
56557 + 2628

59185

In [5]:
temp_df = pd.read_csv('product_feed.csv')
temp_df['PRODUCT TYPE'] = temp_df['PRODUCT TYPE'].str.replace('&gt;', '>')
temp_df['PRODUCT TYPE'] = temp_df['PRODUCT TYPE'].str.replace('&amp;', '&')
temp_df['PRODUCT TYPE'] = temp_df['PRODUCT TYPE'].str.replace('&#039;', "'")
temp_df['TITLE'] = temp_df['TITLE'].str.replace('&gt;', '>')
temp_df['TITLE'] = temp_df['TITLE'].str.replace('&amp;', '&')
temp_df['TITLE'] = temp_df['TITLE'].str.replace('&#039;', "'")
temp_df.rename(columns={'TITLE': 'title'}, inplace=True)
temp_stock_df = temp_df['title'].str.split('\(|\)', expand=True).iloc[:,[0,1]]
temp_df['title'] = temp_stock_df[0]
temp_df['title'] = temp_df['title'].str.lstrip()
temp_df['title'] = temp_df['title'].str.rstrip()
temp_df.drop_duplicates(subset=['title'], inplace=True)
temp_df = temp_df[['title', 'PRODUCT TYPE']]

<>:9: SyntaxWarning: invalid escape sequence '\('
<>:9: SyntaxWarning: invalid escape sequence '\('
C:\Users\retai\AppData\Local\Temp\ipykernel_9244\526015662.py:9: SyntaxWarning: invalid escape sequence '\('
  temp_stock_df = temp_df['title'].str.split('\(|\)', expand=True).iloc[:,[0,1]]


In [6]:
temp_df.applymap(list)['title'].apply(pd.Series).stack().unique()

C:\Users\retai\AppData\Local\Temp\ipykernel_9244\1485600810.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  temp_df.applymap(list)['title'].apply(pd.Series).stack().unique()


array(['L', 'e', 'a', 't', 'h', 'r', ' ', 'J', 'c', 'k', 'B', 'l', '/',
       'n', 'i', ':', 'u', 'o', 'w', 'p', '0', '1', '8', '4', 'd', 'y',
       'M', 'C', 'g', 'I', 'f', 'q', 'G', 'D', 'F', 'H', 'S', 's', '-',
       'A', 'T', 'b', 'v', '2', '7', 'E', 'x', '9', 'Q', 'W', '5', 'm',
       '3', '6', 'V', '&', 'Z', ',', 'N', 'P', 'R', '.', 'O', 'K', 'X',
       'j', 'U', 'z', 'Y', "'", '–', ';', '\xa0', '’'], dtype=object)

In [7]:
temp_df[temp_df['title'] == "Women's Collarless Leather Fashion Jacket Black: AWL-284"]

,title,PRODUCT TYPE
9091,Women's Collarless Leather Fashion Jacket Blac...,Ladies > Leather Jackets > Casual Styles


In [8]:
temp_df['title'].str.contains("Women's Collarless Leather Fashion Jacket Black: AWL-284").any()

True

In [12]:
df1_null = df1_null.merge(temp_df, on='title', how='left')
df1_null['customs_description'] = df1_null['PRODUCT TYPE']
df1_null.drop(['PRODUCT TYPE'], axis=1, inplace=True)
df1_null.shape

(2628, 22)

In [154]:
df1 = pd.concat([df1_notnull, df1_null], ignore_index=True)
df1.drop_duplicates(inplace=True)
df1.shape

(59185, 22)

In [159]:
df2 = df1['customs_description'].str.split(' > ', expand=True)
df1.drop(['Category', 'F_Cat', 'S_Cat', 'T_Cat'], axis=1, inplace=True)
df2.columns = ['F_Cat', 'S_Cat', 'T_Cat']
df1 = df1.join(df2)
df1['Category'] = df1['F_Cat'] + ' > ' + df1['S_Cat']

In [162]:
df1.to_csv('2024_feb_unduplicated_orders fixed_categories.csv')